## List requests
### Lists
There are two rows created from this list as static data. Then the rows also correspond to columns - user_id, reaction_type, post_id. 
```python
emotions = [
        Emotion(user_id=1, reaction_type='😢', title_id="Catcher in the Rye", author_id="J.D."),
        Emotion(user_id=1, reaction_type='❤️', title_id="Hunger Games", author_id="Suzanne Collins")
    ]
```
### Dictionaries
There are three columns generated from dictionary - reaction_type, user_id, and post_id. These three columns 
```python
return {
        "reaction_type": self.reaction_type,
        "user_id": self.user_id,
        "title_id": self.title_id,
        "author_id": self.author_id
    }
```


---
## Function:
This procedure adds a reaction, and has several parameters, including `user_id`, `reaction_type`, `title_id`, and `author_id`.
```python
@emotion_api.route('', methods=['POST']) 
def add_emotion():
    data = request.json

    user_id = data.get("user_id")
    reaction_type = data.get("reaction_type")
    title_id = data.get("title_id")
    author_id = data.get("author_id")

    try:
        # Create and add the message_reaction
        message_emotion = Emotion(reaction_type=reaction_type, user_id=user_id, title_id=title_id, author_id=author_id)
        message_emotion.create()
        return jsonify({'message': 'Emotion added successfully to post'}), 201
    except Exception as e:
        return jsonify({'error': 'Failed to add emotion', 'message': str(e)}), 500
```
---

## Call to Function
```javascript
document.getElementById('addEmotionForm').addEventListener('submit', async (e) => {
    e.preventDefault();

    const userId = document.getElementById('userId').value;
    const bookDropdown = document.getElementById('bookDropdown');
    const selectedOption = bookDropdown.options[bookDropdown.selectedIndex];

    if (!selectedOption.value) {
        showStatusMessage('Please select a book.', false);
        return;
    }

    const bookData = JSON.parse(selectedOption.value);
    const reactionType = document.getElementById('addReactionType').value;

    try {
        // Step 4: Send request to add emotion
        const response = await fetch(`${pythonURI}/api/emotion`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                user_id: userId,
                title_id: bookData.id,
                author_id: bookData.author,
                reaction_type: reactionType,
            }),
        });

        if (!response.ok) throw new Error(await response.text());

        // Step 5: Update UI after successful API response
        displayAddedEmotion(bookData.id, bookData.title, bookData.author, reactionType);
        showStatusMessage('Emotion successfully added!');
    } catch (error) {
        showStatusMessage(`Failed to add emotion: ${error.message}`, false);
    }
});
```


---

# **Project Demo:**
<a href="{{site.baseurl}}/notebooks/tri_2/fullstackcrudrev/"> PPR </a>

The purpose of my feature was to allow users react to books with emojis to show if the book made them feel happy, sad, etc. This is not as quick as a rating, but not quite a review, instead it just allows readers to express how the book made them feel in a simple manner. The program retrieves data from a **backend API** and displays it in an **interactive UI**. Users can:
- Add new items using an input field with an emoji selection feature.
- Update existing items by modifying their content and saving changes.
- Delete items when they are no longer needed.
- The UI dynamically updates based on user actions without requiring a page reload.

A combination of event listeners and API calls **(GET, POST, PUT, DELETE)** ensures that the frontend remains synchronized with the backend.

My project fulfills the CPT requirements of **selection and iteration when adding reaction and updating reactions**. When users delete a reaction, a **for loop iterates through the existing reactions in the database**. My code includes both **inputs and outputs** for when users add reactions, update reactions, etc. 

